In [1]:
# Library Imports
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
from pinocchio import RobotWrapper

import numpy as np
from numpy.linalg import *
np.set_printoptions(precision=4)

import matplotlib.pyplot as plt
plt.style.use('dark_background')

from time import sleep

In [2]:
# Read the .stl files
package_dir = '/home/kanishk/Desktop/Pinocchio-Based-Robot-Solver'
urdf_path = '/home/kanishk/Desktop/Pinocchio-Based-Robot-Solver/model/fingeredu.urdf'

In [3]:
# Build the robot 
model, collision_model, visual_model = pin.buildModelsFromUrdf(urdf_path, package_dir)
data = model.createData()

In [4]:
# Display the visual of the robot
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()
viz.displayCollisions(True)
q0 = pin.neutral(model)
viz.display(q0)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/
<ipython-input-4-055e029a4c68>:5: UserWarning: Plotting collision meshes is not available for MeshcatVisualizer
  viz.displayCollisions(True)


In [5]:
# Get the FrameID of the EOAT
EOAT_ID = model.getFrameId('finger_tip_link')

In [6]:
# Computing Forward Kinematics
def compute_ForwardKinematics(model, data, q):
    pin.framesForwardKinematics(model, data, q)
    pos = np.array(data.oMf[EOAT_ID].translation)
    return pos

In [7]:
# Compute the Jacobian
def compute_Jacobian(model, data, q):
    pin.computeJointJacobians(model, data, q)
    return pin.getFrameJacobian(model, data, EOAT_ID, pin.ReferenceFrame.LOCAL_WORLD_ALIGNED,)[:3]

In [8]:
# Trajectory Planning for Point-to-Point(P2P) Motion
def Trajectory_Planner(model, data, goal_pos, joint_pos, interpolation='linear', smooth=True):
    # Initiate Callers
    q = joint_pos
    y_target = goal_pos
    init_y = compute_ForwardKinematics(model, data, q)
    Error = goal_pos - init_y

    # Constants
    W = 1e-4 * np.eye(3)

    # HyperParams-> Needs to be tuned for large P2P Motion
    if interpolation=='joint':
        max_steps = 100
        smooth = 1e-1
    if interpolation=='linear':
        max_steps = 1000
        smooth = 1

    for i in range(1, max_steps):
        y = compute_ForwardKinematics(model, data, q)
        J = compute_Jacobian(model, data, q) 

        # Compute Error
        Error = goal_pos - y

        if interpolation=='joint':
            pass
        if interpolation=='linear':
            y_target = init_y + (i/max_steps)*(goal_pos-y)

        q += smooth * np.linalg.inv(J.T @ J + W) @ J.T @ (y_target - y)
        viz.display(q)

    return print(f'Positioning Error: {np.linalg.norm(Error)*1e3}mm')

In [9]:
# Point based Trajectory Planner
def TrajPlanner_Complex(model, data, q, iterations):

    # Init. Robot
    w = 1e-4
    W = w * np.identity(3) 
 
    # Make tracepoints for circle
    radius = 0.05
    target_center = compute_ForwardKinematics(model, data, q)
    trace_points = []
    theta = (2 * np.pi)/int(iterations)
    for i in range(int(iterations)+1):
        X = radius*np.cos(theta*i)+target_center[0]
        Z = radius*np.sin(theta*i)+target_center[2]
        trace_points.append(np.array([X, target_center[1], Z]))

    # Trace Circle from center
    for i, point in enumerate(trace_points):
        init_y = compute_ForwardKinematics(model, data, q)
        for j in range(10):
            y = compute_ForwardKinematics(model, data, q)
            J = compute_Jacobian(model, data, q) 
            y_target = init_y + ((j+1)/ 10) * (point - init_y)
            q += inv(J.T @ J + W) @ J.T @ (y_target - y)
            viz.display(q)

In [19]:
# Check Trajectory Generation

# Set the Robot in Random Position
q = np.array([0, -np.pi/3, np.pi/1.5])
viz.display(q)
sleep(1)


# Augment 'goal_pos' from 'curr_pos'
TrajPlanner_Complex(model, data, q, 100)